In [1]:
!pip install pypdf2
!pip install requests
!pip install pycryptodom

DEPRECATION: Loading egg at /home/ubuntu/miniconda3/envs/.venv/lib/python3.11/site-packages/megatron_core-0.2.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 20.1 MB/s eta 0:00:00
DEPRECATION: Loading egg at /home/ubuntu/miniconda3/envs/.venv/lib/python3.11/site-packages/megatron_core-0.2.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /home/ubuntu/miniconda3/envs/.venv/lib/python3.11/site-packages/megatron_core-0.2.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.

In [2]:
!pip install beautifulsoup4
!pip install joblib

DEPRECATION: Loading egg at /home/ubuntu/miniconda3/envs/.venv/lib/python3.11/site-packages/megatron_core-0.2.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.9/147.9 kB 12.0 MB/s eta 0:00:00
DEPRECATION: Loading egg at /home/ubuntu/miniconda3/envs/.venv/lib/python3.11/site-packages/megatron_core-0.2.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [4]:
import requests
import os
from bs4 import BeautifulSoup
from joblib import Parallel, delayed

# クロールするウェブサイトのURL
urls = ["https://data.e-gov.go.jp/data/dataset/?q=報告書&res_format=PDF&res_license_id=cc-by&sort=score+desc%2C+metadata_modified+desc-20",
        "https://data.e-gov.go.jp/data/dataset/?q=報告書&res_format=PDF&res_license_id=gjstu&sort=score+desc%2C+metadata_modified+desc-20",
        "https://data.e-gov.go.jp/data/dataset/?q=報告書&res_format=PDF&res_license_id=CC-BY&sort=score+desc%2C+metadata_modified+desc-20",
        "https://data.e-gov.go.jp/data/dataset/?q=白書&res_format=PDF&res_license_id=cc-by&sort=score+desc%2C+metadata_modified+desc-20",
        "https://data.e-gov.go.jp/data/dataset/?q=白書&res_format=PDF&res_license_id=gjstu&sort=score+desc%2C+metadata_modified+desc-20",
        "https://data.e-gov.go.jp/data/dataset/?q=白書&res_format=PDF&res_license_id=CC-BY&sort=score+desc%2C+metadata_modified+desc-20",]

last_pages = [211, 40, 4, 108, 10, 9]
# ダウンロードするPDFファイルを保存するディレクトリのパス
download_dir = 'pdf_downloads'
if not os.path.exists(download_dir):
    os.makedirs(download_dir)

def crawl_pages(url, last_page=384):
    crawl_search_pdf(url)
    print("page1をクロールしました。")
    for next in range(2, last_page+1):
        next_url = url + "&page=" + str(next)
        crawl_search_pdf(next_url)
        print("page" + str(next) + "をクロールしました。")

def crawl_search_pdf(url):
    """指定されたURLからPDFリンクを見つける"""
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    li_tag = soup.find_all('li', class_='list_dataset_item')
    
    for target in li_tag:
        for link in target.find_all('a', string=['PDF', 'pdf'], href=True):
            href = link.get('href')
            crawl_all_pdfs("https://data.e-gov.go.jp"+href)
    
def crawl_all_pdfs(url):
    """指定されたURLからPDFリンクを見つけてダウンロードする"""
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    li_tag = soup.find_all('li', class_='list_dataset_item resource-item')

    # PDFリンクのリストをリスト内包表記で抽出
    pdf_links = [[link.get('href') for link in target.find_all('a') if link.get('href').endswith('.pdf')] for target in li_tag]
    filtered_pdf_links = [element for element in pdf_links if element]
    Parallel(n_jobs=-1, verbose=0)([delayed(write_pdf_links_to_file)(pdf_link[0]) for pdf_link in filtered_pdf_links])

def write_pdf_links_to_file(pdf_link):
    # PDFリンクをファイルに書き込む
    with open("pdf_downloads/urls.txt", "a", encoding="utf-8") as file:
        file.write(pdf_link + "\n")  # "\n" は新しい行を開始します

# クローリングを開始する
for url, last_page in zip(urls,last_pages):
    crawl_pages(url, last_page=last_page)

page1をクロールしました。


KeyboardInterrupt: 

In [2]:
!cd /home/ubuntu/ucllm_nedo_prod_kawagoshi/data_management

In [2]:
import requests
import PyPDF2
import re
import json
import os
from io import BytesIO

def extract_text_from_pdf_with_pypdf2(url):
    # PDFファイルをダウンロードする
    response = requests.get(url)
    response.raise_for_status()  # HTTPエラーをチェック
    texts = []
    json_text = {}
    # ダウンロードしたPDFファイルをメモリ上で開く
    with BytesIO(response.content) as f:
        # PyPDF2を使用してPDFファイルを読み込む
        pdf_reader = PyPDF2.PdfReader(f)
        text = ""
        
        # 各ページからテキストを抽出
        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            text += page.extract_text() if page.extract_text() else ''
        
    without_kaigyou_text = text.replace('\n', '')
    cleaned_text = re.sub(r'\.\.+', '', without_kaigyou_text)

    json_text['text'] = cleaned_text 
    texts.append(json_text)
    return texts

# 使用例
# download_dir = 'pdf_downloads'
# if not os.path.exists(download_dir):
#     os.makedirs(download_dir)
# url = "https://www.ppc.go.jp/files/pdf/nichibeiou_ekkyouiten_report.pdf"
# texts = extract_text_from_pdf_with_pypdf2(url)
# # JSONLファイルに保存
# with open('pdf_downloads/scraped_data.jsonl', 'a', encoding='utf-8') as file:
#     for data in texts:
#         json_line = json.dumps(data, ensure_ascii=False)  # JSON形式の文字列に変換
#         file.write(json_line + '\n')  
# print(texts[0])  

# urls.txtに保存されたURLからPDFをダウンロードしてテキストを抽出
with open('/home/ubuntu/ucllm_nedo_prod_kawagoshi/data_management/preprocessing/download_dataset/pdf_downloads/urls.txt', 'r', encoding='utf-8') as file:
    urls = file.readlines()
    for url in urls:
        try:
            texts = extract_text_from_pdf_with_pypdf2(url.replace('\n', ''))
            # JSONLファイルに保存
            with open('/home/ubuntu/ucllm_nedo_prod_kawagoshi/data_management/output/datasets/scraped_data.jsonl', 'a', encoding='utf-8') as file:
                for data in texts:
                    json_line = json.dumps(data, ensure_ascii=False)  # JSON形式の文字列に変換
                    file.write(json_line + '\n')
        except Exception as e:
            print(str(e.args) +  url)
            continue

('404 Client Error: Not Found for url: https://www.mhlw.go.jp/sinsei/chotatu/zuikeisinsa/dl/tyoushu_kanjo_0802.pdf',)https://www.mhlw.go.jp/sinsei/chotatu/zuikeisinsa/dl/tyoushu_kanjo_0802.pdf

('404 Client Error: Not Found for url: https://www.mhlw.go.jp/sinsei/chotatu/zuikeisinsa/dl/tyoushu_kanjo_0806.pdf',)https://www.mhlw.go.jp/sinsei/chotatu/zuikeisinsa/dl/tyoushu_kanjo_0806.pdf

('404 Client Error: Not Found for url: https://www.mhlw.go.jp/sinsei/chotatu/zuikeisinsa/dl/tyoushu_kanjo_0903.pdf',)https://www.mhlw.go.jp/sinsei/chotatu/zuikeisinsa/dl/tyoushu_kanjo_0903.pdf

('404 Client Error: Not Found for url: https://www.mhlw.go.jp/sinsei/chotatu/zuikeisinsa/dl/tyoushu_kanjo_0811.pdf',)https://www.mhlw.go.jp/sinsei/chotatu/zuikeisinsa/dl/tyoushu_kanjo_0811.pdf

('404 Client Error: Not Found for url: https://www.mhlw.go.jp/sinsei/chotatu/zuikeisinsa/dl/tyoushu_kanjo_0902.pdf',)https://www.mhlw.go.jp/sinsei/chotatu/zuikeisinsa/dl/tyoushu_kanjo_0902.pdf

('404 Client Error: Not Found 

In [ ]:
def download_pdf(link):
    """PDFファイルをダウンロードする関数"""
    response = requests.get(link)
    filename = link.split('/')[-1]
    with open(os.path.join(download_dir, filename), 'wb') as f:
        f.write(response.content)
    print(f"{filename} をダウンロードしました。")